# Welcome to VizieR

It is a generic notebook, proposed by <a href='http://cds.unistra.fr'>CDS</a><img src='https://cds.unistra.fr//img/cds.png' alt='CDS logo'/>

--------------------------------------------------------

Modify the VizieR catalogue name (variable CATALOGUE)

In [ ]:
CATALOGUE = "J/AJ/157/229"

usefull function to access data

In [ ]:
from pyvo import registry
import matplotlib.pyplot as plt

from astropy import log
log.setLevel('ERROR')

def vizier_ivoid(source: str) ->str:
    return f"ivo://CDS.VizieR/{CATALOGUE}"

def print_basic_info(voresource)-> dict:
    tabinfo = voresource.to_table()
    if len(tabinfo) == 0:
        raise Exception("nothing found")
    info = tabinfo[0]
    print(f"{info['short_name']} {info['res_title']}")
    print(f"Description:\n{info['res_description']}")
    print(f"Bibcode: {info['source_value']}")
    
def get_vizier_table_from_resource(voresource) -> list:
    return list(voresources[0].get_tables().keys())


## Get metadata from the VO registry
the flowing code uses <a href='https://pyvo.readthedocs.io/en/latest/'>pyvo</a>

In [ ]:
# get catalogue information using pyvo
voresources = registry.search(ivoid=vizier_ivoid(CATALOGUE))
print_basic_info(voresources)

Display resource in the browser (open a new target)

In [ ]:
voresources.to_table().show_in_browser()

Print tables available inb the catalogue

In [ ]:
tables = get_vizier_table_from_resource(voresources)
for table in tables:
    print(f"table {table}")

## Execute a simple cone search query 
We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) of the virtual observatory 
(only available for table having position)

In [ ]:
DEFAULT_SCS_RADIUS = 1/60.
DEFAULT_SCS_POS = (0, 0)

In [ ]:
# make a conesearch
print(f"conesearch\n")
records = voresources[CATALOGUE].get_service("conesearch").search(pos=DEFAULT_SCS_POS, sr=DEFAULT_SCS_RADIUS)
print(records)

## Execute a SQL/ADQL query
On the first table of the catalogue, we execute a simple <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query 

In [ ]:
# get the first table of the catalogue
table = list(voresources[0].get_tables().keys())[0]

# execute a synchronous ADQL query
tap_service = voresources[CATALOGUE].get_service("tap")
records = voresources[CATALOGUE].get_service("tap").run_sync(f'select TOP 10 * from "{table}"')
print(records)

## Get the catalogue coverage
We use Multiple Order Coverage <a href='https://ivoa.net/documents/MOC/'>MOC</a> to display the catalogue coverage

In [ ]:
# get moc and print
from mocpy import MOC

def get_moc_url(name: str) -> str:
    return f"https://cdsarc.cds.unistra.fr/viz-bin/moc/{name}"

moc = MOC.from_fits(get_moc_url(CATALOGUE))

plot the result using matplotlib

In [ ]:
fig = plt.figure(figsize=(5, 5))
wcs = moc.wcs(fig)
ax = fig.add_subplot(projection=wcs)
moc.fill(ax=ax, wcs=wcs, alpha=0.5, color="blue")

Print the intersection of the catalogue coverage with <a href='https://cdsarc.cds.unistra.fr/viz-bin/cat/II/335'>Galex</a>

In [ ]:
# get Galex coverage
moc_galex = MOC.from_fits(get_moc_url("II/335"))

# compute intersection
moc_intersect = moc_galex.intersection(moc)

In [ ]:
# display result in matplotlib

fig = plt.figure(figsize=(5, 5))

wcs_galex = moc_galex.wcs(fig)
ax = fig.add_subplot(projection=wcs_galex)
moc_galex.fill(ax=ax, wcs=wcs_galex, alpha=0.5, color="blue")

wcs_intersect = moc_intersect.wcs(fig)
ax_intersect = fig.add_subplot(projection=wcs_intersect)
moc_intersect.fill(ax=ax_intersect, wcs=wcs_intersect, alpha=0.5, color="red")